# Projecting data on a map

In this example, we will see how to use xsar to project data on a map, or to export to geotiff.

In [ ]:
import xsar
import xarray as xr
import numpy as np
import geoviews as gv
gv.extension('bokeh')
xr.set_options(display_expand_data=False)

In [ ]:
filename = xsar.get_test_file('S1B_IW_GRDH_1SDV_20181013T062322_20181013T062347_013130_018428_Z010.SAFE')
s1meta = xsar.Sentinel1Meta(filename)
xsar_obj = xsar.Sentinel1Dataset(s1meta, resolution='100m')

## Reprojecting to epsg 4326, using rioxarray.reproject

[xsar.Sentinel1Dataset.rio](../basic_api.rst#xsar.Sentinel1Dataset.rio) returns a `rioxarray.raster_array.RasterArray` object that we can use with [reproject](https://corteva.github.io/rioxarray/stable/rioxarray.html#rioxarray.raster_array.RasterArray.reproject).

In [ ]:
sigma0_proj = xsar_obj.rio('sigma0').reproject('epsg:4326',shape=(1000,1000),nodata=np.nan).rename({'x': 'Longitude', 'y': 'Latitude'})

In [ ]:
sigma0_image = gv.Image(sigma0_proj.sel(pol='VV')).opts(alpha=0.7, cmap='gray', clim=(0,0.05))
(gv.tile_sources.Wikipedia * sigma0_image).opts(width=600,height=600)

## Exporting to geotiff

### Exporting without colormap

using [rioxarray.raster_array.RasterArray.to_raster](https://corteva.github.io/rioxarray/html/rioxarray.html#rioxarray.raster_array.RasterArray.to_raster), we can save our projected sigma0 to a geotiff

In [ ]:
sigma0_proj.sel(pol='VV').rio.to_raster('/tmp/sigma0_nocolor.tiff')

The geotiff can be read with `gv.load_tiff` but it's stored as float, and we have to pass a cmap. This image cannot be viewed with google earth 

In [ ]:
gv.tile_sources.Wikipedia * gv.load_tiff('/tmp/sigma0_nocolor.tiff').opts(alpha=0.7, cmap='gray', clim=(0,0.05))

### Exporting with RGBA colormap

We have to manually convert the sigma0 floating values to `np.uint8` in the range [0,255]

In [ ]:
from matplotlib import cm
cmap=cm.ScalarMappable(cmap='jet')
cmap.set_clim(vmin=0, vmax=0.05)
rgb_sigma0 = xr.DataArray(
    (cmap.to_rgba(xsar_obj.dataset['sigma0'].sel(pol='VV'), bytes=True)),  
    dims=['atrack', 'xtrack', 'band']
).transpose('band', 'atrack', 'xtrack')
rgb_sigma0


`rgb_sigma0` is now an `xarray.DataArray`, with sames spatials dims `['atrack', 'xtrack']`, and an new dim `band` that hold color in R,G,B,A. 

Note that you must not use rio accessor using `rgb_sigma0.rio`. You have to store this array in `xsar_obj.dataset`, in order to use `xsar_obj.rio('sigma0_rgba')`


In [ ]:
xsar_obj.dataset['sigma0_rgba'] = rgb_sigma0 
xsar_obj.rio('sigma0_rgba').reproject('epsg:4326',shape=(1000,1000),nodata=0).rio.to_raster('/tmp/sigma0_color.tiff')

In [ ]:
# there is a transparency bug in geoviews (https://github.com/holoviz/geoviews/issues/571)
# but if loading this tiff in google earth, it should render properly
(gv.tile_sources.Wikipedia * gv.load_tiff('/tmp/sigma0_color.tiff')).opts(width=600,height=600)

## Reprojecting to pre-defined grid

We now want to project ecmwf data and sigma0 data to the same grid.

The grid is choosen to be 600km*600km , with a spacing of 1km. x and y coords are in **meters**, because we are going to use an **azimuthal equidistant projection**, centered on the xsar dataset

In [ ]:
from rasterio.crs import CRS
from xsar.raster_readers import ecmwf_0100_1h

grid = xr.DataArray(
    np.full((601,601), np.nan), 
    dims=['x','y'], 
    coords={
        'x': np.linspace(-300 * 1000,300 * 1000, 601), 
        'y': np.linspace(-300 * 1000,300 * 1000, 601)
    })


crs = CRS({
        'proj': 'aeqd',
        'lat_0': xsar_obj.s1meta.footprint.centroid.y,
        'lon_0': xsar_obj.s1meta.footprint.centroid.x,
        'x_0': 0,
        'y_0': 0,
        'ellps': 'WGS84'
    })

grid.rio.write_crs(crs, inplace=True)
grid

Now, we can use [reproject_match](https://corteva.github.io/rioxarray/stable/rioxarray.html#rioxarray.raster_array.RasterArray.reproject_match), to project onto `grid`

In [ ]:
sigma0_grid = xsar_obj.rio('sigma0').reproject_match(grid,nodata=np.nan).sel(pol='VV')
ecmwf_ds = ecmwf_0100_1h(xsar.get_test_file('ECMWF_FORECAST_0100_202109091300_10U_10V.nc'))
ecmwf_ds['spd'] = np.sqrt(ecmwf_ds['U10']**2+ecmwf_ds['V10']**2)
ecmwf_grid = ecmwf_ds['spd'].rio.reproject_match(grid,nodata=np.nan)

# ecmwf_grid and sigma0_grid has the same shape and the same coordinates, we are able to merge them

merged_grid = xr.merge([sigma0_grid, ecmwf_grid],combine_attrs='drop')
merged_grid

geoviews and cartopy are not able to handle azimuthal equidistant projection, so we use `holoviews` here.

(But note that it will display properly in google earth, if exported to geotiff)

In [ ]:
import holoviews as hv
hv.extension('bokeh')
hv.Image(merged_grid['spd']).opts(cmap='jet') * hv.Image(merged_grid['sigma0']).opts(cmap='gray', clim=(0,0.05), tools=['hover']) 

To use holoviews, we can reproject to lon/lat

In [ ]:
merged_lonlat = merged_grid.rio.write_crs(crs).rio.reproject(4326, nodata=np.nan).rename({'x': 'Longitude', 'y': 'Latitude'})
(gv.tile_sources.Wikipedia * gv.Image(merged_lonlat['spd']).opts(cmap='jet') * gv.Image(merged_lonlat['sigma0']).opts(cmap='gray', clim=(0,0.05))).opts(width=600,height=600)
                                                           